In [192]:
import hashlib
import json
import datetime

In [193]:
class Block:
    # a block is a set of data, if you send someone ETH, the transactional data needs to be added to the block
    # a block could be seen as a container that holds a list of transactions, such as sending and receiving money
    def __init__(self, index, previous_hash, transactions, timestamp):
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.timestamp = timestamp

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "previous_hash": self.previous_hash,
            "transactions": self.transactions,
            "timestamp": self.timestamp
        })
        return hashlib.sha256(block_string.encode("utf_8")).hexdigest()
    

# add NONCE=0 for initial transaction?
genesis_hash = '0x88e96d4537bea4d9c05d12549907b32561d3bf31f45aae734cdc119f13406cb6'
transactional_data = {
    't1':{'from': 'darcy', 'to':'bingle', 'amount':25},
    't2':{'from': 'lydia', 'to':'jingle', 'amount':10}
}
timestamp_unix = datetime.datetime.timestamp(datetime.datetime.now())
block1 = Block(index=1, previous_hash=initial_hash, transactions=transactional_data, timestamp=timestamp_unix)
block1.calculate_hash()

'0ed2d3eaad690add58e404bd71b26ab8d9b63edd625c9166c04c31ff5d653bde'

In [194]:
transactional_data = {}
timestamp_unix = datetime.datetime.timestamp(datetime.datetime.now())
block2 = Block(index=1, previous_hash=block1.calculate_hash(), transactions=transactional_data, timestamp=timestamp_unix)

In [195]:
# https://youtu.be/_160oMzblY8
# each peer has the exact same copy of the blockchain with all the same values
# if you have just 1 peer, they can mutate one block and mine it to verify the block, i.e., produce new hashes until they get the one they want
# but if you have 3 peers, all with identical and verified blocks, one can verify it, but then it will be different to peers - condigtion not met and rejected
# blockchain could have 400k - 500k blocks

In [196]:
class Blockchain:
    # blocks are linked together in chronological order to form a chain
    # each block cryptographically references parent
    def __init__(self):
        self.chain = []

    def add_block(self, block):
        self.chain.append(block)

    def get_previous_block(self):
        return self.chain[-1]

    def is_valid_block(self, block):
        if block.index != self.get_previous_block().index + 1:
            return False

        if block.previous_hash != self.get_previous_block().calculate_hash():
            return False

        transaction_hashes = []
        for transaction in block.transactions:
            transaction_hashes.append(transaction.calculate_hash())

        if block.calculate_hash() != sorted(transaction_hashes)[-1]:
            return False

        return True

In [197]:
class Transaction:
    def __init__(self, sender, recipient, amount):
            self.sender = sender
            self.recipient = recipient
            self.amount = amount
            self.hash = None

    def calculate_hash(self):
        transaction_string = json.dumps({
            "sender": self.sender,
            "recipient": self.recipient,
            "amount": self.amount
        })
        return hashlib.sha256(transaction_string.encode('utf-8')).hexdigest()

In [198]:
# create blockchain
eth_bc = Blockchain()

In [199]:
# create first transaction
genesis_block = Block(index=0, previous_hash=initial_hash, transactions=transactional_data, timestamp=datetime.datetime.timestamp(datetime.datetime.now()))
genesis_block.hash = genesis_block.calculate_hash()
eth_bc.add_block(genesis_block)

In [200]:
# create second transaction
t1 = Transaction('alice', 'bob', 10)
block1 = Block(1, genesis_block.hash, t1.calculate_hash(), datetime.datetime.timestamp(datetime.datetime.now()))
block1.hash = block1.calculate_hash()
eth_bc.add_block(block1)

In [206]:
eth_bc.chain[1].__dict__

{'index': 1,
 'previous_hash': '2416362cdecf11db852e37f631cdf421d4bc079957b0181a6fd5363aaf66edf7',
 'transactions': '9067ec49ed4cb73fc148c2ee03ed01eecc9a9d5a4dd41fc9466bee705ae3cddd',
 'timestamp': 1701114263.224768,
 'hash': 'd594edc9c4639905d31e982eb34f697cdd46fc9d3d9fbdfa5d9c6e2ea90bca3b'}

In [202]:
for block in eth_bc.chain:
    print(block.__dict__)

{'index': 0, 'previous_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'transactions': {}, 'timestamp': 1701114263.215181, 'hash': '2416362cdecf11db852e37f631cdf421d4bc079957b0181a6fd5363aaf66edf7'}
{'index': 1, 'previous_hash': '2416362cdecf11db852e37f631cdf421d4bc079957b0181a6fd5363aaf66edf7', 'transactions': '9067ec49ed4cb73fc148c2ee03ed01eecc9a9d5a4dd41fc9466bee705ae3cddd', 'timestamp': 1701114263.224768, 'hash': 'd594edc9c4639905d31e982eb34f697cdd46fc9d3d9fbdfa5d9c6e2ea90bca3b'}


In [ ]:
# create a simple blockchain program that:
# has an interface allowing you to select number of peers, number of blocks,  number of transactions 
# simlates transaction data 
# simulate manipulation of a block for personal gain, and run verification procedures
# simulate mining in python 
# block creator, MEV in python, strategic positiong of transactions within a block to generate a profit - arbitrage or priority transactions
# proof of stake example: